In [1]:
from glob import glob
import librosa
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import random
import os
import gc
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation


from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr
from libs import spectrogram as spec
from libs import criterion as cr

import warnings
warnings.filterwarnings("ignore")

/home/yuigahama/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
train_tp = pd.read_csv('../../data/train_tp.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]


SR = 48000
LENGTH = SR * 6

In [3]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore
set_seed(53)

In [4]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [5]:
"""class RfcxDataSet(Dataset):
    def __init__(self,
                 df:pd.DataFrame,
                 data_path:str,
    ):
        self.df =  df
        self.path = data_path
        self.img_size = 256
        
        self.transform = tr.Compose([
          tr.OneOf([
            tr.GaussianNoiseSNR(min_snr=10),
            tr.PinkNoiseSNR(min_snr=10)
          ]),
          #tr.PitchShift(max_steps=2, sr=SR),
          #tr.TimeStretch(),
          #tr.TimeShift(sr=SR),
          #tr.VolumeControl(mode="sine")
        ])
        
    
    def __len__(self):
        return len(self.df)
    
    def get_duration(self, t_min, t_max):
        annotated_duration = t_max - t_min
        
        if annotated_duration > 10:
            limit_sec = t_max - 10
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + 10
            
            start = start_sec * SR
            end = end_sec * SR
        else:
            res_time = 10 - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start = (t_min - front_time) * SR
            end = (t_max + back_time) * SR
            
        return start, end
    
    def get_no_call(self, t_min, t_max):
        t_min = int(t_min * 0.9)
        t_max = int(t_max * 1.1)
        
        back_time = 60 - t_max
        
        if t_min > back_time:
            limit = t_min - 11
            start = random.randint(0, limit)
        else:
            limit = 50
            start = random.randint(t_max+1, limit)
        end = start + 10
        
        return start * SR, end * SR
    
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        species_id = sample['species_id']
        is_no_call = sample['is_nocall']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        record_path = self.path + recording_id + '.flac'
        y, sr = librosa.load(record_path, sr=None)
        
        random_limit = y.shape[0] - LENGTH
        
        start = random.randint(0, random_limit)
        end = start + LENGTH
        
        target = torch.zeros([25], dtype=torch.float32)
        
        if is_no_call:
            start, end = self.get_no_call(t_min, t_max)
            target[-1] = 1
        else:
            start, end = self.get_duration(t_min, t_max)
            target[species_id] = 1

        tarnsform_flag = random.choice([True, True, False])
        
        if tarnsform_flag:
            y = self.transform(y)
        
        
        return y[start:end], target""";

In [6]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 df:pd.DataFrame,
                 data_path:str,
    ):
        self.df =  df
        self.path = data_path
        self.img_size = 256
        
        self.transform = tr.Compose([
          tr.OneOf([
            tr.GaussianNoiseSNR(min_snr=10),
            tr.PinkNoiseSNR(min_snr=10)
          ]),
          tr.PitchShift(max_steps=2, sr=SR),
          #tr.TimeStretch(),
          #tr.TimeShift(sr=SR),
          tr.VolumeControl(mode="sine")
        ])
        
    
    def __len__(self):
        return len(self.df)
    
    def get_duration(self, t_min, t_max, duration=10):
        annotated_duration = t_max - t_min
        
        if annotated_duration > duration:
            limit_sec = t_max - duration
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + duration
            
            start = start_sec * SR
            end = end_sec * SR
        else:
            res_time = duration - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start = (t_min - front_time) * SR
            end = (t_max + back_time) * SR
            
        return start, end
    
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        species_id = sample['species_id']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        record_path = self.path + recording_id + '.flac'
        y, sr = librosa.load(record_path, sr=None)
    
        start, end = self.get_duration(t_min, t_max, 8)
        
        target = torch.zeros([25], dtype=torch.float32)
        target[species_id] = 1
        
        y = self.transform(y)
        
        random_limit = y.shape[0] - LENGTH
        start = random.randint(0, random_limit)
        end = start + LENGTH
        
        
        return y[start:end], target

In [7]:
class RfcxTestDataSet(Dataset):
    def __init__(self,
                 df:pd.DataFrame,
                 data_path:str,
                 val=False
    ):
        self.df =  df
        self.val = val
        self.path = data_path
        self.img_size = 256
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        record_path = self.path + recording_id + '.flac'
        
        y, sr = librosa.load(record_path, sr=None)
        
        images = []
        
        start = 0
        for i in range(10):
            end = start + (SR * 6)
            images.append(y[start:end])
            start = end
            
        if self.val == True:
            species_id = sample['species_id']
            target = torch.zeros([25], dtype=torch.float32)
            target[species_id] = 1
            return images, target
        else:
            return images

In [8]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)
    
def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

class AttBlockV2(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [9]:
class BaseModel(nn.Module):
    def __init__(self, model_type, model_name, output_size, spectrogram_params, logmel_extractor_params, spec_augmenter_params, pce_params):
        super().__init__()
        self.model_type = model_type
        
        self.spectrogram_extractor = Spectrogram(**spectrogram_params)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(**logmel_extractor_params)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(**spec_augmenter_params)
        
        #Pcen converter
        self.pcen_converter = spec.pcen(**pce_params)
        
        self.bn0 = nn.BatchNorm2d(logmel_extractor_params['n_mels'])
        
        self.mixup_alpha = 0.2
        self.random_state = np.random.RandomState(123)
        
        if model_type == 'res':
            if model_name == 'resnet50':
                base_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
            else:
                base_model = resnet34(pretrained=True)
            
            layers = list(base_model.children())[:-2]
            self.encoder = nn.Sequential(*layers)
            self.interpolate_ratio = 30

            in_features = base_model.fc.in_features
            
            self.att_block = AttBlockV2(in_features, output_size, activation="sigmoid")

            self.fc1 = nn.Linear(in_features, in_features, bias=True)
            
            """self.fc1 = nn.Sequential(
                nn.Linear(in_features, 1024),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(1024, 1024),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(1024, output_size)
            )"""

            
        elif model_type == 'efficientnet':
            self.features = EfficientNet.from_pretrained(model_name)
            model_code = model_name.split('-')[1]
            
            num_input = {
                'b4': 1792,
                'b2': 1408,
                'b7': 2560
            }
            
            self.features_out = num_input[model_code]
            
            self.classification = nn.Sequential(nn.Linear(num_input[model_code], output_size))
            
    def mixup(self, x):
        lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
        index = list(range(x.size(0)))
        random.shuffle(index)
        out = (x * lam + x[index].squeeze() * (1-lam))
        
        return out, {'lam': lam, 'index': index}

    def norm_spec(self, x):
        batch_size = x.size(0)
        height = x.size(2)
        width = x.size(3)
        
        x = x.squeeze().view(batch_size, -1)
        
        x -= x.min(1, keepdim=True)[0]
        x /= x.max(1, keepdim=True)[0]
        x = x.view(batch_size, height, width)
        
        return x.unsqueeze(1)
        
    def forward(self, input):
        if self.model_type == 'res':
            x = self.spectrogram_extractor(input)
            x = self.logmel_extractor(x)
            
            x_mels = self.norm_spec(x)
            x_pcen = self.norm_spec(self.pcen_converter(x))
            x_clear = self.norm_spec(self.logmel_extractor.power_to_db(x ** 1.5))
            
            #x = torch.stack([self.norm_spec(x_mels), x_pcen, x_clear], dim=1).squeeze()
            x = torch.stack([x_mels, x_pcen, x_clear], dim=1).squeeze()
            
            frames_num = x.shape[2]
            
            x = x.transpose(1, 3)
            x = self.bn0(x)
            x = x.transpose(1, 3)
            
            """if self.training:
                x, mix_info = self.mixup(x)
                x = self.spec_augmenter(x)
            else:
                mix_info = None"""
            
            x = self.encoder(x)
            
            x = torch.mean(x, dim=2)

            # channel smoothing
            x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
            x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
            x = x1 + x2
            
            x = F.dropout(x, p=0.5, training=self.training)
            
            x = x.transpose(1, 2)
            x = F.relu_(self.fc1(x))
            x = x.transpose(1, 2)
            
            x = F.dropout(x, p=0.5, training=self.training)
            (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
            logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
            segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
            segmentwise_output = segmentwise_output.transpose(1, 2)

            # Get framewise output
            framewise_output = interpolate(segmentwise_output,
                                           self.interpolate_ratio)
            framewise_output = pad_framewise_output(framewise_output, frames_num)

            framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
            framewise_logit = pad_framewise_output(framewise_logit, frames_num)

            output_dict = {
                "framewise_output": framewise_output,
                "segmentwise_output": segmentwise_output,
                "logit": logit,
                "framewise_logit": framewise_logit,
                "clipwise_output": clipwise_output
            }

            return output_dict   
                        
        elif self.model_type == 'efficientnet':
            x = self.features.extract_features(input)
            x = F.avg_pool2d(x, x.size()[2:]).reshape(-1, self.features_out)
            x = self.classification(x)
            
            return x, mix_info

In [10]:
def save(fold, model, optim, criterion, file_path="../../model/"):
    if not TEST_NAME in os.listdir(file_path):
        os.mkdir(file_path+TEST_NAME)
    
    
    output_path = file_path + TEST_NAME + '/' + f"{TEST_NAME}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [11]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()

# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

def mixup_socre(cor, x, y, mix_info):
    return cor(x, y) * mix_info['lam'] + cor(x, y[mix_info['index']].squeeze()) * (1-mix_info['lam'])

# Sample usage
# y_true = torch.tensor(np.array([[1, 1, 0], [1, 0, 1], [0, 0, 1]]))
# y_score = torch.tensor(np.random.randn(3, 3))
# print(LRAP(y_score, y_true), LWLRAP(y_score, y_true))

In [12]:
TEST_NAME = 'baseline-pic-res'

n_splits = 5
random_state = 1
epochs = 55

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spectrogram_params = {
    'n_fft': 2048,
    'hop_length': 512,
    'win_length': 1024,
    'window': 'hann',
    'center': True,
    'pad_mode': 'reflect',
    'freeze_parameters': True
}

logmel_extractor_params = {
    'sr': SR,
    'n_fft': 2048,
    'n_mels': 80,
    'fmin': 0,
    'fmax': 14000,
    'ref' : 1.0,
    'amin': 1e-10,
    'top_db': None,
    'is_log': False,
    'freeze_parameters': True
}

spec_augmenter_params = {
    'time_drop_width':  64,
    'time_stripes_num': 2,
    'freq_drop_width':  8,
    'freq_stripes_num': 2
}

pce_params = {
    'gain': 0.98,
    'bias': 2,
    'power': 0.5,
    'time_constant': 0.4,
    'eps': 0.000001,
}

model_params = {
    'model_type': 'res', # res or efficientnet
    'model_name': 'resnet50',
    'output_size': 25,
    'spectrogram_params': spectrogram_params,
    'logmel_extractor_params': logmel_extractor_params,
    'spec_augmenter_params': spec_augmenter_params,
    'pce_params': pce_params
}

optim_params = {
    'lr': 1e-3,
    'weight_decay': 5e-5,
    'betas': (0.9, 0.999)
}

scheduler_params = {
    'mode': 'max',
    'patience': 1,
    'factor': 0.6,
    'verbose': False
}

data_params = {
    'data_path': '/home/yuigahama/kaggle/rfcx/data/train/',
}

test_data_params = {
    'data_path': '/home/yuigahama/kaggle/rfcx/data/test/',
}

dataloder_params = {
    'batch_size': 15,
    'num_workers': 15,
    'pin_memory': False,
}

test_dataloder_params = {
    'batch_size': 32,
    'num_workers': 15,
    'pin_memory': False,
    'shuffle':False
}

In [13]:
tta = np.zeros((len(submission), 24))
cv_score = 0

for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp.species_id)):
    print(f'---------- fold {fold_id} ----------')
    
    model = BaseModel(**model_params).to(device)
    optim = Adam(model.parameters(), **optim_params)
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optim, **scheduler_params)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optim, T_max=epochs)
    
    pos_weights = torch.ones(25)
    pos_weights = pos_weights * 25
    criterion_ = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    criterion = cr.ImprovedFocalLoss(weights=[0.5, 1])
    
    train_dataset = RfcxDataSet(train_tp.iloc[train_index], **data_params)
    val_dataset   = RfcxTestDataSet(train_tp.iloc[val_index], val=True, **data_params)
    test_dataset  = RfcxTestDataSet(submission, **test_data_params)

    train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloder_params)
    val_dataloader = DataLoader(val_dataset, shuffle=False, **dataloder_params)
    test_dataloader = DataLoader(test_dataset, **test_dataloder_params)
    
    es = 15
    for epoch in range(1, epochs):
        if es == 0:
            break
        
        start_time = time.time()
        bast_score = 0
        
        # train
        model.train()
        train_loss = 0
        train_score = 0
        
        for data in train_dataloader:
            image = data[0].float().to(device)
            label = data[1]
            
            output = model(image)
            
            optim.zero_grad()
            
            output = {k:v.cpu() for k,v in output.items()}
            #loss = mixup_socre(criterion, pred.cpu(), label.cpu(), mix_info)
            #score = mixup_socre(LWLRAP, torch.sigmoid(pred.cpu()), label.cpu(), mix_info)
            #loss = criterion(output['clipwise_output'].cpu(), label)
            loss = criterion(output, label)
            score = LWLRAP(output['clipwise_output'].cpu(), label)
            
            loss.backward()
            optim.step()
            
            train_loss += loss.item()
            train_score += score
            
        train_loss  /= len(train_dataloader)
        train_score /= len(train_dataloader)
        
        # val
        model.eval()
        val_loss = 0
        val_score = 0
        
        with torch.no_grad():
            for val_data in val_dataloader:
                num = len(val_data[0])
                label = val_data[1]
                tmp = torch.zeros((val_data[0][0].size(0), 25))
                
                for img in val_data[0]:
                    output = model(img.to(device))
                    tmp += output['clipwise_output'].cpu() / num
                
                val_loss += criterion_(tmp, label).item()
                val_score += LWLRAP(tmp, label)

                
        val_loss  /= len(val_dataloader)
        val_score /= len(val_dataloader)
        
        duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
        print(f'epoch {epoch:3}| T | loss: {train_loss:.3} | score: {train_score:.3} | V | loss: {val_loss:.3} | score: {val_score:.3} | time: {duration}')

        if bast_score < val_score:
            bast_score = val_score
            bast_path = save(fold_id, model, optim, criterion)
        else:
            es -= 1
        
        scheduler.step(val_score)
            
        
    oof = np.zeros((len(test_dataset), 24))
    model = BaseModel(**model_params).to(device)
    
    bast_model_parms = torch.load(bast_path)
    model.load_state_dict(bast_model_parms['model_state_dict'])

    
    with torch.no_grad():
        start = 0
        for i, test_data in enumerate(test_dataloader):
            target_size = test_data[0].size(0)
            tmp_pred = np.zeros((target_size, 24))
            num = len(test_data)
            end = start + target_size
            for img in test_data:
                img = img.to(device)
                output = model(img)
                
                tmp_pred += output['clipwise_output'].cpu().detach().numpy()[:, :-1] / num
                       
            oof[start:end, :] = tmp_pred
            start = end
            
    tta += (oof / n_splits)
    
    del model, train_dataset, val_dataset, test_dataset, train_dataloader, val_dataloader, test_dataloader
    del bast_model_parms, optim
    gc.collect()

---------- fold 0 ----------


Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-dc31c0fdfad3>", line 38, in <module>
    for data in train_dataloader:
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 435, in __next__
    data = self._next_data()
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1068, in _next_data
    idx, data = self._get_data()
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1034, in _get_data
    success, data = self._try_get_data()
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 872, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
  File "/home/yuigahama/anaconda3/lib/python3.8/multiprocessing/

TypeError: object of type 'NoneType' has no len()

In [ ]:
print(cv_score)

In [ ]:
pred_sub = pd.DataFrame(tta, columns=pred_target)
pred_sub

In [ ]:
pd.concat([submission['recording_id'], pred_sub], axis=1).to_csv('submission.csv', index=False)